<a href="https://colab.research.google.com/github/arivv22/tomato-leaf-disease-detection-tf/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup & Import

In [2]:
#if you already instal just skip this code
!pip install -q tensorflow matplotlib scikit-learn pandas seaborn opencv-python-headless gradio

In [4]:
import os, sys, math, random, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# Load data json

In [ ]:
!pip install kaggle

In [24]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d emmarex/plantdisease
!unzip -q plantdisease.zip -d plantvillage


# Data Preparation

In [37]:
# Filter only Tomato classes
import os, shutil, pathlib
base_dir = pathlib.Path('plantvillage/PlantVillage') #adjust with your path dataset
tomato_dir = pathlib.Path('tomato_dataset')
tomato_dir.mkdir(exist_ok=True)
for folder in base_dir.iterdir():
    if 'Tomato' in folder.name:
        shutil.copytree(folder, tomato_dir/folder.name, dirs_exist_ok=True)
print('✅ Tomato classes:', [f.name for f in tomato_dir.iterdir()])

✅ Tomato classes: ['Tomato__Target_Spot', 'Tomato_Septoria_leaf_spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Late_blight', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_healthy', 'Tomato_Early_blight', 'Tomato_Leaf_Mold', 'Tomato__Tomato_mosaic_virus', 'Tomato_Bacterial_spot']


In [38]:
import pathlib
DATA_ROOT = pathlib.Path(tomato_dir)

all_subdirs = [p.name for p in DATA_ROOT.iterdir() if p.is_dir()]
tomato_dirs = [d for d in all_subdirs if d.lower().startswith("tomato")]

print("Detected tomato folders (examples):", tomato_dirs[:10])

# Create a working dataset folder with train/val/test splits
WORKDIR = 'tomato_dataset'
if os.path.exists(WORKDIR):
    shutil.rmtree(WORKDIR)
os.makedirs(WORKDIR, exist_ok=True)

def make_split(src_root, dest_root, classes, val_ratio=0.15, test_ratio=0.10, seed=42):
    random.seed(seed)
    for c in classes:
        src = pathlib.Path(src_root) / c
        if not src.exists(): continue
        imgs = list(src.glob('*.jpg')) + list(src.glob('*.png'))
        random.shuffle(imgs)
        n = len(imgs)
        n_test = int(n * test_ratio)
        n_val = int(n * val_ratio)
        splits = {
            'train': imgs[n_val + n_test:],
            'val': imgs[:n_val],
            'test': imgs[n_val:n_val + n_test]
        }
        for split, files in splits.items():
            dst_dir = pathlib.Path(dest_root) / split / c
            dst_dir.mkdir(parents=True, exist_ok=True)
            for f in files:
                shutil.copy(f, dst_dir / f.name)

make_split(DATA_ROOT, WORKDIR, tomato_dirs)
print("Done. New dataset at:", WORKDIR)


Detected tomato folders (examples): ['Tomato__Target_Spot', 'Tomato_Septoria_leaf_spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Late_blight', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_healthy', 'Tomato_Early_blight', 'Tomato_Leaf_Mold', 'Tomato__Tomato_mosaic_virus', 'Tomato_Bacterial_spot']
Done. New dataset at: /content/tomato_dataset
